In [4]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd
import datetime

In [2]:
tokenizer = TapasTokenizer.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base-finetuned-wtq')

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [28]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
print(cfs_data.shape)
cfs_data.sample(5)

The number of incidents in the set is 8802
(8802, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
2546,220061353,E36,22002799,Medical Incident,2022-01-06T00:00:00.000,2022-01-06T00:00:00.000,2022-01-06T11:41:14.000,2022-01-06T11:42:44.000,2022-01-06T11:42:52.000,2022-01-06T11:44:52.000,...,1,ENGINE,1,2.0,5,Hayes Valley,220061353-E36,"{'type': 'Point', 'coordinates': [-122.4251974...",NaN,NaN
7667,220110719,E05,22005209,Structure Fire,2022-01-11T00:00:00.000,2022-01-11T00:00:00.000,2022-01-11T08:40:51.000,2022-01-11T08:40:51.000,2022-01-11T08:41:35.000,2022-01-11T08:42:55.000,...,1,ENGINE,5,2,6,Tenderloin,220110719-E05,"{'type': 'Point', 'coordinates': [-122.4121906...",NaN,NaN
2912,220060068,E29,22002579,Alarms,2022-01-06T00:00:00.000,2022-01-05T00:00:00.000,2022-01-06T00:44:40.000,2022-01-06T00:46:30.000,2022-01-06T00:47:07.000,2022-01-06T00:48:36.000,...,1,ENGINE,1,10.0,10,Potrero Hill,220060068-E29,"{'type': 'Point', 'coordinates': [-122.3986985...",NaN,NaN
1182,220052624,E35,22002427,Medical Incident,2022-01-05T00:00:00.000,2022-01-05T00:00:00.000,2022-01-05T17:57:47.000,2022-01-05T18:01:03.000,2022-01-05T18:04:16.000,2022-01-05T18:06:04.000,...,1,ENGINE,1,3,6,Financial District/South Beach,220052624-E35,"{'type': 'Point', 'coordinates': [-122.3932397...",NaN,NaN
6705,220100270,T07,22004612,Alarms,2022-01-10T00:00:00.000,2022-01-09T00:00:00.000,2022-01-10T04:10:32.000,2022-01-10T04:12:01.000,2022-01-10T04:12:26.000,2022-01-10T04:14:14.000,...,1,TRUCK,2,6,9,Mission,220100270-T07,"{'type': 'Point', 'coordinates': [-122.4183512...",NaN,NaN


In [32]:
cfs_data=cfs_data.dropna()
cfs_data= cfs_data.astype(str)
cfs_data=cfs_data.head(500)
cfs_data_tapas=cfs_data[['call_date','call_type','neighborhoods_analysis_boundaries']]

In [33]:
queries = ["When was the most recent medical incident?", "How many incidents are in Tenderloin?"]

In [34]:
inputs = tokenizer(table=cfs_data, queries=queries, padding="max_length", return_tensors="pt")

IndexError: iloc cannot enlarge its target object

In [35]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"]
}
table = pd.DataFrame.from_dict(data)
queries = ["How many movies has George Clooney played in?", "How old is Brad Pitt?"]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
logits_aggregation = outputs.logits_aggregation

In [36]:
outputs

TableQuestionAnsweringOutput(loss=None, logits=tensor([[ -9951.5186,  -9951.5186,  -9951.5186,  ...,  -9951.5186,
          -9951.5186,  -9951.5186],
        [-10042.0918, -10042.0918, -10042.0918,  ..., -10042.0918,
         -10042.0918, -10042.0918]], grad_fn=<ViewBackward0>), logits_aggregation=tensor([[ -8.4079,  -9.4974, -10.9121,  12.9523],
        [ -1.2786, -14.6947,  16.3172, -18.6797]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [38]:
logits

tensor([[ -9951.5186,  -9951.5186,  -9951.5186,  ...,  -9951.5186,
          -9951.5186,  -9951.5186],
        [-10042.0918, -10042.0918, -10042.0918,  ..., -10042.0918,
         -10042.0918, -10042.0918]], grad_fn=<ViewBackward0>)